In [2]:
!pip install transformers
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
2024-04-28 17:46:39.611296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 17:46:39.882866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-28 17:46:39.882913: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore

In [ ]:
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import operator
import nltk
import math
from scipy.stats import norm
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
import random
import spacy
from nltk.corpus import wordnet
from nltk.tag import pos_tag

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-28 17:43:27.048877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 17:43:27.243732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-28 17:43:27.243759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202

In [ ]:
import pandas as pd
import numpy as np 
from IPython.display import display
import os

In [ ]:
!python -m nltk.downloader punkt

/usr/local/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
adjudications = pd.read_csv("adjudications.csv") # has 544 data points
adjudications = adjudications.sample(frac=1, random_state=11).drop(columns=['data point ID', 'adjudicated'])
training = adjudications[:327] # training with 327 rows has around 60% of the 544 data points
development = adjudications[327:436] # development with 109 rows has around 20% of the 544 data points
test = adjudications[436:544] # test with 108 rows has around 20% of the 544 data points
training

,label,text
369,witty,"We’re renovating the house, and the first floo..."
372,funny,Why do peppers make such good archers? Because...
175,funny,My wife gave me an ultimatum: Her or my addict...
538,lame,What did the ravioli play on his birthday? Pas...
347,lame,If Trump wanted to avoid impeachment... ...he ...
...,...,...
479,witty,Do you know how many people are dead at a ceme...
384,funny,"My mother in law bought a talking parrot, but ..."
478,lame,"Denmark: ""We will kill 17 million minks by 202..."
289,witty,Where do you learn all about ice cream? Sundae...


In [ ]:
# make txt files 
training.to_csv("train.tsv", sep="\t", index=False)
os.rename("train.tsv", "train.txt")

development.to_csv("dev.tsv", sep="\t", index=False)
os.rename("dev.tsv", "dev.txt")

test.to_csv("test.tsv", sep="\t", index=False)
os.rename("test.tsv", "test.txt")

In [ ]:
training

,label,text
369,witty,"We’re renovating the house, and the first floo..."
372,funny,Why do peppers make such good archers? Because...
175,funny,My wife gave me an ultimatum: Her or my addict...
538,lame,What did the ravioli play on his birthday? Pas...
347,lame,If Trump wanted to avoid impeachment... ...he ...
...,...,...
479,witty,Do you know how many people are dead at a ceme...
384,funny,"My mother in law bought a talking parrot, but ..."
478,lame,"Denmark: ""We will kill 17 million minks by 202..."
289,witty,Where do you learn all about ice cream? Sundae...


In [ ]:
trainX = training['text']
trainY = training['label']
devX = development['text']
devY = development['label']
testX = test['text']
testY = test['label']

# LOGISTIC REGRESSION

In [ ]:
#Logistic Regression 

class Classifier:

    def __init__(self, feature_method, trainX, trainY, devX, devY, testX, testY):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.min_feature_count=2
        self.log_reg = None

        self.trainY=trainY
        self.devY=devY
        self.testY=testY
        
        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):
        
        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    # Train model and evaluate on held-out data
    def train(self):
        (D,F) = self.trainX.shape
        best_dev_accuracy=0
        best_model=None
        for C in [0.1, 1, 10, 100]:
            self.log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
            self.log_reg.fit(self.trainX, self.trainY)
            training_accuracy = self.log_reg.score(self.trainX, self.trainY)
            development_accuracy = self.log_reg.score(self.devX, self.devY)
            if development_accuracy > best_dev_accuracy:
                best_dev_accuracy=development_accuracy
                best_model=self.log_reg

#             print("C: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (C, training_accuracy, development_accuracy))

        self.log_reg=best_model
        

    def test(self):
        return self.log_reg.score(self.testX, self.testY)
        

    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

In [ ]:
def binary_bow_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        feats[word]=1
            
    return feats

In [ ]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [ ]:
simple_classifier = Classifier(binary_bow_featurize, trainX, trainY, devX, devY, testX, testY)
simple_classifier.train()
accuracy=simple_classifier.test()
accuracy

0.5

# MAJORITY CLASS CLASSIFIER

In [ ]:
#Majority Class Classifier
def majority_class(trainY, evalY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority_class=labelCounts.most_common(1)[0][0]
    
    correct=0.
    for label in evalY:
        if label == majority_class:
            correct+=1
    return majority_class, correct/len(evalY)

mc, mc_devAcc=majority_class(trainY, devY)


print("Majority class: %s, dev accuracy: %.3f" % (mc, mc_devAcc))

Majority class: witty, dev accuracy: 0.743


# FEATURIZATIONS!!!!

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# featurizations

def multiple_meanings(text): 
    '''
    one possible feature for pun detection.
    this function looks for multiple meanings in the words in a given text.
    '''

    features = {}
    tokens = nltk.word_tokenize(text)

    for token in tokens:
      token = token.lower()
      synsets = wordnet.synsets(token)
      if token not in features.keys():
          features[token] = len(synsets)

    return features
    
def culture_references(text): 
    '''
    a possible feature to detect whether a joke makes general and/or pop culture references.
    primarily utilizes named entity recognition (NER).
    '''
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    named_entities = [ent.text for ent in doc.ents]

    features = {}
    tokens = nltk.word_tokenize(text) 

    for token in tokens:
        if token in named_entities:
            features[token] = 1
        else:
            features[token] = 0

    return features 

def expected_pos(text): # food for thought
    # detects whether a word in a text is the expected part of speech
    # if not, this may be indicative of the word's purpose as a pun 
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

def word_count(text):
    '''
    classifies as 'witty' or not-'witty' based on word count.
    Based on EDA, we find that the avg word count for 'witty' label is ~14.7, while 'funny' is ~18.9 and 'lame' is ~19.8.
    Utilize BoW kinda not rly
    '''
    witty_count = 14.7
    funny_count = 18.9
    lame_count = 19.8

    witty_class

    features = {}
    tokens = nltk.word_tokenize(text) 

    features[tokens] = len(tokens)

def lame_list(text):
    '''
    classifies as 'lame' if topic of joke contains a word from a list of words
    Utilizes BoW
    '''
    lame_topics = ['onlyfans', 'trump', 'dick', 'nut', 'corona', 'covid', 'covid-19', 'date', 'depresso', 'sex'] 

    features = {}
    tokens = nltk.word_tokenize(text) 

    for word in words:
        word.lower()
        if word in lame_topics:
            features[tokens] = 1
        else:
            features[token] = 0    

In [ ]:
# consolidate all features into one function to feed into our classifier later 
def combined_features(text):
    all_features={}
    for feature in [multiple_meanings, culture_references, word_count, lame_list]:
        all_features.update(feature(text))
    return all_features

# Classifier with features

In [ ]:
#This function takes in an array of texts, and for each text, finds the number of 
#words that have multiple meanings 

def count_multiple_meanings(texts):
    meanings_count = np.array([])
    for text in texts:
        meanings = list(multiple_meanings(text).values())
        meanings_count = np.append(meanings_count, sum([word > 1 for word in meanings]))
    return meanings_count

train_meanings = count_multiple_meanings(trainX)
dev_meanings = count_multiple_meanings(devX)
test_meanings = count_multiple_meanings(testX)

In [ ]:
def count_references(texts):
    reference_count = np.array([])
    for text in texts:
        references = list(culture_references(text).values())
        reference_count = np.append(reference_count, sum(references))
    return reference_count

#count_references(trainX)

# BERT

In [ ]:
# now let's try running our features through a BERT classifier
# see https://github.com/ucbnlp24/hws4nlp24/blob/main/AP/BERT.ipynb for inspo 

In [ ]:
# trainX = training['text']
# trainY = training['label']
# devX = development['text']
# devY = development['label']
# testX = test['text']
# testY = test['label']

trainX.tolist()
trainY.tolist()

#trainX, trainY, devX, devY, testX, testYtrainX, trainY, devX, devY, testX, testY

['witty',
 'funny',
 'funny',
 'lame',
 'lame',
 'lame',
 'witty',
 'witty',
 'funny',
 'witty',
 'lame',
 'lame',
 'witty',
 'witty',
 'lame',
 'funny',
 'lame',
 'lame',
 'witty',
 'funny',
 'lame',
 'witty',
 'witty',
 'witty',
 'lame',
 'funny',
 'witty',
 'witty',
 'funny',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'witty',
 'witty',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'funny',
 'lame',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'witty',
 'funny',
 'witty',
 'funny',
 'witty',
 'funny',
 'witty',
 'witty',
 'lame',
 'witty',
 'witty',
 'lame',
 'lame',
 'funny',
 'lame',
 'witty',
 'funny',
 'lame',
 'lame',
 'witty',
 'lame',
 'funny',
 'lame',
 'lame',
 'witty',
 'lame',
 'lame',
 'lame',
 'witty',
 'witty',
 'lame',
 'lame',
 'witty',
 'funny',
 'funny',
 'witty',
 'funny',
 'lame',
 'witty',
 'witty',
 'witty',
 'lame',
 'funny',
 'witty',
 'funny',
 'witty',
 'witty',
 '

In [ ]:
def read_labels(col):
    labels={}
    for elem in col: 
        if elem not in labels:
            labels[elem] = len(labels)
    return labels

In [ ]:
def read_data(x, y, labels, max_data_points=1000):
  
    data = []
    data_labels = []

    for text, label in zip(x, y):
        data.append(text)
        data_labels.append(labels[label])
            

    # shuffle the data
    tmp = list(zip(data, data_labels))
    random.shuffle(tmp)
    data, data_labels = zip(*tmp)
    
    if max_data_points is None:
        return data, data_labels
    
    return data[:max_data_points], data_labels[:max_data_points]

In [ ]:
labels = read_labels(trainY)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_x, train_y=read_data(trainX, trainY, labels, max_data_points=None)
dev_x, dev_y=read_data(devX, devY, labels, max_data_points=None)
test_x, test_y=read_data(testX, testY, labels, max_data_points=None)

In [3]:
class BERTClassifier(nn.Module):

    def __init__(self, bert_model_name, params):
        super().__init__()
    
        self.model_name=bert_model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=params["doLowerCase"], do_basic_tokenize=False)
        self.bert = BertModel.from_pretrained(self.model_name)
        
        self.num_labels = params["label_length"]

        self.fc = nn.Linear(params["embedding_size"], self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=16, max_toks=510):
            
        """ Get batches for input x, y data, with data tokenized according to the BERT tokenizer 
      (and limited to a maximum number of WordPiece tokens """

        batches_x=[]
        batches_y=[]
        
        for i in range(0, len(all_x), batch_size):

            current_batch=[]

            x=all_x[i:i+batch_size]

            batch_x = self.tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=max_toks)
            batch_y=all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))
            
        return batches_x, batches_y
  

    def forward(self, batch_x): 
    
        bert_output = self.bert(input_ids=batch_x["input_ids"],
                         attention_mask=batch_x["attention_mask"],
                         token_type_ids=batch_x["token_type_ids"],
                         output_hidden_states=True)

      # We're going to represent an entire document just by its [CLS] embedding (at position 0)
      # And use the *last* layer output (layer -1)
      # as a result of this choice, this embedding will be optimized for this purpose during the training process.
      
        bert_hidden_states = bert_output['hidden_states']

        out = bert_hidden_states[-1][:,0,:]

        out = self.fc(out)

        return out.squeeze()

NameError: name 'nn' is not defined

In [ ]:
def train(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=None):

    bert_model = BERTClassifier(bert_model_name, params={"label_length": len(labels), "doLowerCase":doLowerCase, "embedding_size":embedding_size})
    bert_model.to(device)

    batch_x, batch_y = bert_model.get_batches(train_x, train_y)
    dev_batch_x, dev_batch_y = bert_model.get_batches(dev_x, dev_y)

    optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)
    cross_entropy=nn.CrossEntropyLoss()

    num_epochs=30
    best_dev_acc = 0.
    patience=5

    best_epoch=0

    for epoch in range(num_epochs):
        bert_model.train()

        # Train
        for x, y in zip(batch_x, batch_y):
            y_pred = bert_model.forward(x)
            loss = cross_entropy(y_pred.view(-1, bert_model.num_labels), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate
        dev_accuracy, _=evaluate(bert_model, dev_batch_x, dev_batch_y)
        if epoch % 1 == 0:
            print("Epoch %s, dev accuracy: %.3f" % (epoch, dev_accuracy))
            if dev_accuracy > best_dev_acc:
                torch.save(bert_model.state_dict(), model_filename)
                best_dev_acc = dev_accuracy
                best_epoch=epoch
        if epoch - best_epoch > patience:
            print("No improvement in dev accuracy over %s epochs; stopping training" % patience)
            break

    bert_model.load_state_dict(torch.load(model_filename))
    print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))
    return bert_model

In [ ]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [ ]:
# train bert model 
bert_model_name="bert-base-cased"
model_filename="mybert.model"
embedding_size=768
doLowerCase=False

model=train(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
test_batch_x, test_batch_y = model.get_batches(test_x, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
text = "I want to go on record that I support farming. As a matter of fact, you could call me protractor."
tokens = nltk.word_tokenize(text)
pos_tags = pos_tag(tokens, tagset='universal')
pos_tags


#IM LOSING IT

[('I', 'PRON'),
 ('want', 'VERB'),
 ('to', 'PRT'),
 ('go', 'VERB'),
 ('on', 'ADP'),
 ('record', 'NOUN'),
 ('that', 'ADP'),
 ('I', 'PRON'),
 ('support', 'VERB'),
 ('farming', 'VERB'),
 ('.', '.'),
 ('As', 'ADP'),
 ('a', 'DET'),
 ('matter', 'NOUN'),
 ('of', 'ADP'),
 ('fact', 'NOUN'),
 (',', '.'),
 ('you', 'PRON'),
 ('could', 'VERB'),
 ('call', 'VERB'),
 ('me', 'PRON'),
 ('protractor', 'NOUN'),
 ('.', '.')]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3b6f8c22-749e-426f-b9e2-c8df176f9f68' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>